In [1]:
# snp scratchpad initiation
from helper import *
from snp_func import *

# Do assignments
a = assign_var('snp')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

ib =  get_connected('snp', 'live')

In [ ]:
opts(ib)

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [2]:
import pandas as pd
dft = pd.read_pickle('../data/snp/base.pickle')

In [3]:
util.logToFile(logpath+'test.log')

In [5]:
syms = ['VXX','P', 'TSRO', 'GOOGL'] + list({p.contract.symbol for p in ib.positions()})

In [7]:
df_mega=dft.sort_values('symbol').reset_index(drop=True)
df_mega = df_mega[~df_mega.symbol.isin(syms)].reset_index(drop=True)

In [ ]:
# get the max fall and rise
df_frs = df_mega.drop('strike', 1).drop_duplicates() # for fall rise std

mfrs = [((s, d) + get_maxfallrisestd(ib=ib, c=c, dte=d, tradingdays=tradingdays, durmult=durmult)) for s, c, d in zip(df_frs.symbol, df_frs.contract, df_frs.dte)]

df_mfrs = pd.DataFrame(mfrs, columns=['symbol', 'dte', 'lo52', 'hi52', 'Fall', 'Rise', 'Std'])

df_mega = df_mega.set_index(['symbol', 'dte']).join(df_mfrs.set_index(['symbol', 'dte'])).reset_index()
df_mega['loStd'] = df_mega.undPrice - df_mega['Std'].multiply(putstdmult)
df_mega['hiStd'] = df_mega.undPrice + df_mega['Std'].multiply(callstdmult)

# flter puts and calls by standard deviation
df_puts = df_mega[df_mega.strike < df_mega.loStd]
df_calls = df_mega[df_mega.strike > df_mega.hiStd]

df_puts = df_puts.assign(right='P')
df_calls = df_calls.assign(right='C')

# qualify the options
df = pd.concat([df_puts, df_calls]).reset_index(drop=True).drop('contract', axis=1)

# qualify the option contracts
optipl = [Option(s, e, k, r, exchange) for s, e, k, r in zip(df.symbol, df.expiration, df.strike, df.right)]
optblks = [optipl[i: i+blk] for i in range(0, len(optipl), blk)] # blocks of optipl
cblks = [ib.qualifyContracts(*s) for s in optblks]
contracts = [z for x in cblks for z in x]

# prepare the qualified options dataframe
dfq = util.df(contracts).iloc[:, 1:6]
dfq.columns=['optId', 'symbol', 'expiration', 'strike', 'right'] # rename columns
dfq = dfq.set_index(['symbol', 'expiration', 'strike', 'right']) # set index

# join the qualified options with main dataframe. Remove those without conId
df_opt = df.set_index(['symbol', 'expiration', 'strike', 'right']).join(dfq).dropna()

# convert optId to int for price and margin
df_opt = df_opt.assign(optId=df_opt.optId.astype('int32'))

# get the prices
opt_tickers = ib.reqTickers(*contracts)
optPrices = {t.contract.conId: t.marketPrice() for t in opt_tickers}

df_opt = df_opt.assign(optPrice=df_opt.optId.map(optPrices))
df_opt = df_opt[~(df_opt.optPrice <= 0)].reset_index() # remove options without a price

#... get the margins
# {conId: contract} dictionary
c_dict = {c.conId: c for c in contracts if c.conId in list(df_opt.optId)}
orders = [Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True)]*len(c_dict)

# get the margins
margins = [ib.whatIfOrder(c, o).initMarginChange for c, o in zip(c_dict.values(), orders)]

df_opt = df_opt.assign(optMargin = margins)
df_opt = df_opt.assign(optMargin = abs(pd.to_numeric(df_opt.optMargin).astype('float'))) # convert to float
df_opt = df_opt[df_opt.optMargin < 1e7] # remove options with very large margins

# get the rom
df_opt = df_opt.assign(rom=df_opt.optPrice/df_opt.optMargin*365/df_opt.dte).sort_values('rom', ascending=False)

df_opt.to_pickle(fspath+'opts.pickle')